In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
pd.options.display.max_columns = 999

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from sklearn.neighbors import KDTree
from collections import Counter

import operator
from math import *

In [2]:
df = pd.read_csv('./data/autos.csv')
df1 = pd.read_csv('./data/autos.csv')

In [3]:
def clean(df, get_time=False):
    df = df.dropna(subset=['model', 'fuelType', 'vehicleType', 'gearbox', 
                           'notRepairedDamage', 'offerType', 'nrOfPictures'])
    df['used_month'] = (2017 - df['yearOfRegistration']) * 12 + (4 - df['monthOfRegistration'])
    features = df.columns.values
    df = df.drop_duplicates(features[i] for i in [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15])
    df = df[(df.price >= 10)
            & (df.price <= 1e6)
            & (df.used_month >= 0)
            & (df.used_month <= 500)            
            & (df.powerPS >= 30)
            & (df.powerPS <= 800)]
    df = df.drop(['yearOfRegistration', 'monthOfRegistration', 'seller'], axis=1)
    df.gearbox = df.gearbox.map({'manuell':1, 'automatik':0})
    df.notRepairedDamage = df.notRepairedDamage.map({'ja':1, 'nein':0})
    df.abtest = df.abtest.map({'test':1, 'control':0}) 
    if get_time:
        df_time = df[['dateCreated', 'lastSeen', 'dateCrawled', 'name']]
        df = df.drop(['dateCreated', 'lastSeen', 'dateCrawled', 'offerType'], axis=1)
        return df, df_time
    else:
        return df

In [4]:

def in_list(list1, list2):
    p = True
    for i in list1:
        p = p & (i in list2)
    return p

def gearbox_filter(df, gearbox):
    titles = ['manuell', 'automatik']
    if gearbox == titles[0]:
        print 'Successful'
        return df[df.gearbox == 1]
    elif gearbox == titles[1]:
        print 'Successful'
        return df[df.gearbox == 0]
    else:
        print 'No such choice, Not processing'
        return df
    
def notRepairedDamage_filter(df, notRepairedDamage):
    titles = ['ja', 'nein']
    if notRepairedDamage == titles[0]:
        print 'Successful'
        return df[df.notRepairedDamage == 1]
    elif notRepairedDamage == titles[1]:
        print 'Successful'
        return df[df.notRepairedDamage == 0]
    else:
        print 'No such choice, Not processing'
        return df
    
def abtest_filter(df, abtest):
    titles = ['test', 'control']
    if abtest == titles[0]:
        print 'Successful'
        return df[df.abtest == 1]
    elif abtest == titles[1]:
        print 'Successful'
        return df[df.abtest == 0]
    else:
        print 'No such choice, Not processing'
        return df
    
def price_filter(df, max_price, min_pirce):
    if max_price < 1e6 and min_pirce > 10:
        print 'Successful'
        return df[(df.price < max_price) & (df.price > min_pirce)]
    else:
        print 'Out of price range (10 - 1e6), Not processing'
        return df
    
def kilometer_filter(df, max_kilometer, min_kilometer):
    if max_kilometer < 150000 and min_kilometer > 1e4:
        print 'Successful'
        return df[(df.kilometer < max_kilometer) & (df.kilometer > min_kilometer)]
    else:
        print 'Out of kilometer range (1e4 - 1.5e5), Not processing'
        return df
    
def powerPS_filter(df, max_powerPS, min_powerPS):
    if max_powerPS < 800 and min_powerPS > 30:
        print 'Successful'
        return df[(df.powerPS < max_powerPS) & (df.powerPS > min_powerPS)]
    else:
        print 'Out of powerPS range (30 - 800), Not processing'
        return df
    
def fuelType_filter(df, fuelTypes):
    titles = ['benzin', 'diesel', 'lpg', 'andere', 'hybrid', 'cng', 'elektro']
    if in_list(fuelTypes, titles):
        print 'Successful'
        dff = df.copy().drop(range(len(df)))
        for i in fuelTypes:
            dff = pd.concat([dff, df[df.fuelType == i]], axis=0)
        return dff
    else:
        print 'Out of fuelType (benzin, diesel, lpg, andere, hybrid, cng, elektro), Not processing'
        return df
    
def brand_filter(df, brands):
    titles = ['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'porsche', 'toyota', 'chevrolet',
       'dacia', 'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar',
       'daewoo', 'rover', 'land_rover', 'lada']
    if in_list(brands, titles):
        print 'Successful'
        dff = df.copy().drop(range(len(df)))
        for i in brands:
            dff = pd.concat([dff, df[df.brands == i]], axis=0)
        return dff
    else:
        print 'Out of brand'
        print 'brand : ', titles
        print 'Not processing'
        return df   
    
def usedmonthes_filter(df, max_usedmonthes, min_usedmonthes):
    if max_usedmonthes < 500 and min_usedmonthes > 4:
        print 'Successful'
        return df[(df.used_month < max_usedmonthes) & (df.used_month > min_usedmonthes)]
    else:
        print 'Out of used_month range (4 - 500), Not processing'
        return df

In [5]:
def filter(df, gearbox_f=False, notRepairedDamage_f=False, abtest_f=False, price_f=False,
          kilometer_f=False, powerPS_f=False, fuelType_f=False, brand_f=False, usedmonthes_f=False):
    if gearbox_f: 
        print '-'*50
        print 'All gearbox types:', df1.gearbox.unique()
        gearbox = raw_input('Input the type of gearbox: ')
        df = gearbox_filter(df, gearbox)
        print '-'*50
    if notRepairedDamage_f: 
        print '-'*50
        print 'All gearbox types:', df1.notRepairedDamage.unique()
        gearbox = raw_input('Input the type of notRepairedDamage: ')
        df = notRepairedDamage_filter(df, notRepairedDamage)
        print '-'*50
    if abtest_f: 
        print '-'*50
        print 'All abtest types:', df1.abtest.unique()
        gearbox = raw_input('Input the type of abtest: ')
        df = abtest_filter(df, abtest)
        print '-'*50
    if price_f: 
        print '-'*50
        print 'The price range:', df.price.min(), 'to', df.price.max()
        min_price = input('Input minimum price: ')
        max_price = input('Input maximum price: ')
        df = price_filter(df, max_price, min_price)
        print '-'*50
    if kilometer_f: 
        print '-'*50
        print 'The kilometer range:', df.kilometer.min(), 'to', df.kilometer.max()
        min_kilometer = input('Input minimum kilometer: ')
        max_kilometer = input('Input maximum kilometer: ')
        df = kilometer_filter(df, max_kilometer, min_kilometer)
        print '-'*50
    if powerPS_f: 
        print '-'*50
        print 'The powerPS range:', df.powerPS.min(), 'to', df.powerPS.max()
        min_powerPS = input('Input minimum powerPS: ')
        max_powerPS = input('Input maximum powerPS: ')
        df = powerPS_filter(df, max_powerPS, min_powerPS)
        print '-'*50
    if fuelType_f: 
        print '-'*50
        print 'All fuelType types:', df1.fuelType.unique()
        fuelType = raw_input('Input the type of fuelType: ')
        df = fuelType_filter(df, fuelType)
        print '-'*50
    if brand_f: 
        print '-'*50
        print 'All brand types:', df1.brand.unique()
        brand = raw_input('Input the type of brand: ')
        df = brand_filter(df, brand)
        print '-'*50
    if usedmonthes_f: 
        print '-'*50
        print 'The usedmonthes range:', df1.used_month.min(), 'to', df1.used_month.max()
        min_usedmonthes = input('Input minimum usedmonthes: ')
        max_usedmonthes = input('Input maximum usedmonthes: ')
        df = kilometer_filter(df, max_usedmonthes, min_usedmonthes)
    return df

In [6]:
def dummy(df, features):
    for feature in features:
        uni_words = []
        for word in df[feature].values:
            if word is not np.nan:
                uni_words.append(word)
        uni_words = list(set(uni_words))
        for word in uni_words:
            df[feature + '_' + word] = df[feature].str.contains(word).astype(float)
        df = df.drop(feature, axis=1)
    return df

In [7]:
df, df_time = clean(df, get_time=True)

dummy_feature = ['brand', 'vehicleType', 'model', 'fuelType']
df = filter(df, gearbox_f=True, fuelType_f=True, price_f=True)
df = dummy(df, dummy_feature)
df1 = clean(df1, get_time=False)

--------------------------------------------------
All gearbox types: ['manuell' 'automatik' nan]
Input the type of gearbox: manuell
Successful
--------------------------------------------------
--------------------------------------------------
The price range: 10 to 1000000
Input minimum price: 3000
Input maximum price: 10000
Successful
--------------------------------------------------
--------------------------------------------------
All fuelType types: ['benzin' 'diesel' nan 'lpg' 'andere' 'hybrid' 'cng' 'elektro']
Input the type of fuelType: benzin
Out of fuelType (benzin, diesel, lpg, andere, hybrid, cng, elektro), Not processing
--------------------------------------------------


In [8]:
name_encoder = LabelEncoder()
name_encoder.fit(df.name)
ndf = df.copy()
ndf.name = name_encoder.transform(df.name)

In [9]:
df.head()

name  price  abtest  gearbox  powerPS  \
4       Skoda_Fabia_1.4_TDI_PD_Classic   3600       1        1       69   
18    Ford_C_MAX_2.0_TDCi_DPF_Titanium   7550       1        1      136   
27      Honda_Civic_1.4_i_VTEC_Comfort   6900       1        1       99   
43  Mini_One_Pepper_Scheckheftgepflegt   6990       1        1       95   
46                        BMW_120d_DPF   7999       0        1      177   

    kilometer  notRepairedDamage  nrOfPictures  postalCode  used_month  \
4       90000                  0             0       60437         105   
18     150000                  0             0       88361         118   
27      60000                  0             0       12621         101   
43     100000                  0             0       59174         116   
46     150000                  0             0       53604         116   

    brand_mini  brand_ford  brand_skoda  brand_seat  brand_trabant  \
4          0.0         0.0          1.0         0.0            0.0   
18         0.0         1.0          0.0         0.0            0.0   
27         0.0         0.0          0.0         0.0            0.0   
43         1.0         0.0          0.0         0.0            0.0   
46         0.0         0.0          0.0         0.0            0.0   

    brand_subaru  brand_mercedes_benz  brand_land_rover  brand_volvo  \
4            0.0                  0.0               0.0          0.0   
18           0.0                  0.0               0.0          0.0   
27           0.0                  0.0               0.0          0.0   
43           0.0                  0.0               0.0          0.0   
46           0.0                  0.0               0.0          0.0   

    brand_opel  brand_audi  brand_rover  brand_chevrolet  brand_porsche  \
4          0.0         0.0          0.0              0.0            0.0   
18         0.0         0.0          0.0              0.0            0.0   
27         0.0         0.0          0.0              0.0            0.0   
43         0.0         0.0          0.0              0.0            0.0   
46         0.0         0.0          0.0              0.0            0.0   

    brand_suzuki  brand_smart  brand_dacia  brand_lada  brand_honda  \
4            0.0          0.0          0.0         0.0          0.0   
18           0.0          0.0          0.0         0.0          0.0   
27           0.0          0.0          0.0         0.0          1.0   
43           0.0          0.0          0.0         0.0          0.0   
46           0.0          0.0          0.0         0.0          0.0   

    brand_hyundai  brand_volkswagen  brand_mazda  brand_bmw  brand_kia  \
4             0.0               0.0          0.0        0.0        0.0   
18            0.0               0.0          0.0        0.0        0.0   
27            0.0               0.0          0.0        0.0        0.0   
43            0.0               0.0          0.0        0.0        0.0   
46            0.0               0.0          0.0        1.0        0.0   

    brand_mitsubishi  brand_peugeot  brand_daewoo  brand_fiat  brand_jaguar  \
4                0.0            0.0           0.0         0.0           0.0   
18               0.0            0.0           0.0         0.0           0.0   
27               0.0            0.0           0.0         0.0           0.0   
43               0.0            0.0           0.0         0.0           0.0   
46               0.0            0.0           0.0         0.0           0.0   

    brand_renault  brand_jeep  brand_alfa_romeo  brand_nissan  brand_toyota  \
4             0.0         0.0               0.0           0.0           0.0   
18            0.0         0.0               0.0           0.0           0.0   
27            0.0         0.0               0.0           0.0           0.0   
43            0.0         0.0               0.0           0.0           0.0   
46            0.0         0.0               0.0           0.0           0.0   

    brand_d

In [10]:
ndf.head()

name  price  abtest  gearbox  powerPS  kilometer  notRepairedDamage  \
4   27972   3600       1        1       69      90000                  0   
18  10312   7550       1        1      136     150000                  0   
27  13532   6900       1        1       99      60000                  0   
43  18613   6990       1        1       95     100000                  0   
46   4071   7999       0        1      177     150000                  0   

    nrOfPictures  postalCode  used_month  brand_mini  brand_ford  brand_skoda  \
4              0       60437         105         0.0         0.0          1.0   
18             0       88361         118         0.0         1.0          0.0   
27             0       12621         101         0.0         0.0          0.0   
43             0       59174         116         1.0         0.0          0.0   
46             0       53604         116         0.0         0.0          0.0   

    brand_seat  brand_trabant  brand_subaru  brand_mercedes_benz  \
4          0.0            0.0           0.0                  0.0   
18         0.0            0.0           0.0                  0.0   
27         0.0            0.0           0.0                  0.0   
43         0.0            0.0           0.0                  0.0   
46         0.0            0.0           0.0                  0.0   

    brand_land_rover  brand_volvo  brand_opel  brand_audi  brand_rover  \
4                0.0          0.0         0.0         0.0          0.0   
18               0.0          0.0         0.0         0.0          0.0   
27               0.0          0.0         0.0         0.0          0.0   
43               0.0          0.0         0.0         0.0          0.0   
46               0.0          0.0         0.0         0.0          0.0   

    brand_chevrolet  brand_porsche  brand_suzuki  brand_smart  brand_dacia  \
4               0.0            0.0           0.0          0.0          0.0   
18              0.0            0.0           0.0          0.0          0.0   
27              0.0            0.0           0.0          0.0          0.0   
43              0.0            0.0           0.0          0.0          0.0   
46              0.0            0.0           0.0          0.0          0.0   

    brand_lada  brand_honda  brand_hyundai  brand_volkswagen  brand_mazda  \
4          0.0          0.0            0.0               0.0          0.0   
18         0.0          0.0            0.0               0.0          0.0   
27         0.0          1.0            0.0               0.0          0.0   
43         0.0          0.0            0.0               0.0          0.0   
46         0.0          0.0            0.0               0.0          0.0   

    brand_bmw  brand_kia  brand_mitsubishi  brand_peugeot  brand_daewoo  \
4         0.0        0.0               0.0            0.0           0.0   
18        0.0        0.0               0.0            0.0           0.0   
27        0.0        0.0               0.0            0.0           0.0   
43        0.0        0.0               0.0            0.0           0.0   
46        1.0        0.0               0.0            0.0           0.0   

    brand_fiat  brand_jaguar  brand_renault  brand_jeep  brand_alfa_romeo  \
4          0.0           0.0            0.0         0.0               0.0   
18         0.0           0.0            0.0         0.0               0.0   
27         0.0           0.0            0.0         0.0               0.0   
43         0.0           0.0            0.0         0.0               0.0   
46         0.0           0.0            0.0         0.0               0.0   

    brand_nissan  brand_toyota  brand_daihatsu  brand_saab  brand_chrysler  \
4            0.0           0.0             0.0         0.0             0.0   
18           0.0           0.0             0.0         0.0             0.0   
27           0.0           0.0             0.0         0.0             0.0   
43           0.0           0.0             0.0   

In [11]:
# import time
# def name_filter(df, names):
#     mdf = df.copy()
#     start_time = time.time()
#     index = []
#     uni_index = []
#     titles = name_encoder.inverse_transform(mdf.loc[:200].name)
#     for name in names:
#         for i in titles:
#             if name.lower() in i.lower():
#                 index.append(mdf[mdf.name == name_encoder.transform([str(i)])[0]].index.values)
    
#     for i in xrange(len(index)):
#         for j in xrange(len(index[i])):

#             uni_index.append(index[i][j])
#     uni_index = set()
#     mdf = mdf.loc[uni_index]
#     print 'running time:', time.time() - start_time
#     return mdf

In [12]:
data = ndf.drop_duplicates('name').drop('name', axis=1)
data = MinMaxScaler().fit_transform(data)
tree = KDTree(data, leaf_size=2)
titles = name_encoder.inverse_transform(ndf.drop_duplicates('name').name)


In [13]:
def get_cars(names, titles, k=10):
    cars = []
    for name in names:
        for i in titles:
            if name.lower() in i.lower():
                cars.append(i)
    carcodes = name_encoder.transform(cars)
    unique, counts = np.unique(np.array(carcodes), return_counts=True)
    code = dict(zip(unique, counts))

    sorted_code = sorted(code.items(), key=operator.itemgetter(1))[::-1][:k]
    high_carcode = [sorted_code[i][0] for i in xrange(len(sorted_code))]
    high_cars = name_encoder.inverse_transform(high_carcode)
    return high_cars, code


In [14]:
def recommend(cars, tree, titles, data):
    titles = list(titles)
    recommendations, valid_index = [], []
    
    for i, car in enumerate(cars):
        dist, index = tree.query([data[titles.index(car)]], k=5)
        for d, m in zip(dist[0], index[0]):
            recommendations.append((d, titles[m]))
    rec_code = sorted(recommendations, key=operator.itemgetter(0), reverse=False)
    rec = [i[1].strip() for i in rec_code if i[1] not in cars]
    
    for i in xrange(len(rec_code)):
            if rec_code[i][1] not in cars:
                valid_index.append(i)
    rec_code1 = [i for j, i in enumerate(rec_code) if j in valid_index]

    return rec, rec_code1

In [15]:
print '-'*50
names = raw_input('Input the key words: ').split(',')
print '-'*50
cars, code = get_cars(names, titles, k=int(ceil(30. / len(names))))
rec, rec_code = recommend(cars, tree, titles, data)

print 'Rank | Used cars', '| All %s cars' %(len(rec))
print '-----|', '-'*44
fmt = '{}.   | {}'
for index, car in enumerate(rec[:9]):
    print(fmt.format(index + 1, car))
    
fmt = '{}.  | {}'
for index, car in enumerate(rec[10:min(31, len(rec))]):
    print(fmt.format(index + 10, car))

--------------------------------------------------
Input the key words: golf,sport,edition
--------------------------------------------------
Rank | Used cars | All 38 cars
-----| --------------------------------------------
1.   | Volkswagen_Golf_1.6_Sport_Edition__17"Santa_Monica__T�V_neu
2.   | Volkswagen_Golf_IV_1.6_Sport_Edition__Leder__T�V_11/2017_...
3.   | Volkswagen_Golf_1.6_Sport_Edition__17"Santa_Monica__T�V_neu
4.   | Volkswagen_Golf_1.9_TDI_Comfortline_T�V_neu
5.   | Volkswagen_Golf_1.9_TDI_Comfortline_Euro_4
6.   | Volkswagen_Golf_1.9_TDI_schekheftgepflegt_T�V_bis_10/17
7.   | Golf4_1_9_TDI_Gruene_Plakette
8.   | Volkswagen_Golf_1.9_TDI_18Zoll_Eibach_Gewindefahrwerk
9.   | Volkswagen_Golf_V1.9_TDI_Comfortline__Schecksh._Top_Zustand
10.  | Golf_4___1.4___75PS___Klima__SHD__Alarm
11.  | Vw_Golf_1.9_TDI/_Gruene_Plakette
12.  | Volkswagen_Golf_TDI_GTI
13.  | Volkswagen_Golf_1.9_TDI_VOLLERLEDER
14.  | Vw_golf_IV_1.6_Fsi_16v_Neue_T�V_!!!
15.  | Volkswagen_Golf_1.9_TDI_Edition_E

In [16]:
degree, name, index_df1 = [], [], []
for i in xrange(len(rec_code)):
    degree.append(rec_code[i][0])
    name.append(rec_code[i][1])
df.loc[df['name'].isin(name)]

name  price  abtest  \
25017                         Volkswagen_Golf_1.6_Edition   3200       1   
25635   Volkswagen_Golf_1.6_Sport_Edition__17"Santa_Mo...   3420       0   
30794                         Volkswagen_Golf_1.6_Edition   3300       1   
36784                     Volkswagen_Golf_1.9_TDI_Edition   3590       0   
39166                      VW_Golf_6_2.0_TDI_R_Line_Model   7150       1   
43426                 Vw_golf_IV_1.6_Fsi_16v_Neue_T�V_!!!   3150       1   
59278   Volkswagen_Golf_2.0_TDI_Comfortline_NAVI_Leder...   6999       0   
61175   Volkswagen_Golf_1.9_TDI_Edition_Euro_4_Klimaau...   3450       0   
64157                         Volkswagen_Golf_1.6_Edition   6590       1   
67805   Volkswagen_Golf_2.0_TDI_DPF_Standheiz._Xenon_V...   7500       0   
71035              Volkswagen_Golf_V_service_vor__7000_km   3500       0   
71243                VW_Golf_VI_1_4_TSI_Highline_mit_Navi   9950       0   
72520                                      VW_Golf_IV_TDI   3300       0   
85431                            Volkswagen_Golf_1.4_NAVI   3400       1   
86477   Volkswagen_Golf_VI_1.2_TSI_Team_schwarz__Parkp...   9200       0   
90020                    Vw_Golf_1.9_TDI/_Gruene_Plakette   3500       0   
98612   Volkswagen_Golf_V1.9_TDI_Comfortline__Schecksh...   4100       1   
111129                        Volkswagen_Golf_1.6_Edition   3700       0   
116778                    Volkswagen_Golf_1.9_TDI_Edition   3390       1   
123608                        Volkswagen_Golf_1.6_Edition   7350       1   
125126                    Volkswagen_Golf_1.9_TDI_Edition   3650       1   
129615                      Vw_Golf_6_18_Zoll_xenon_Leder   9500       0   
140294        Golf_4_IV_1.9TDI_Edition_AHK_KLIMAAUTOMATIK   3333       0   
141216                        Volkswagen_Golf_1.6_Edition   7940       0   
146623                            Volkswagen_Golf_TDI_GTI   3499       0   
147699            Golf_4___1.4___75PS___Klima__SHD__Alarm   3300       0   
168678                    Volkswagen_Golf_1.9_TDI_Edition   3999       1   
173650                        Volkswagen_Golf_1.6_Edition   8300       1   
181646                    Volkswagen_Golf_1.9_TDI_Edition   3900       1   
187979                      Golf4_1_9_TDI_Gruene_Plakette   3200       0   
192593                        Volkswagen_Golf_1.6_Edition   3500       0   
195073         Volkswagen_Golf_1.9_TDI_Comfortline_Euro_4   3900       1   
197577  Volkswagen_Golf_IV_1.6_Sport_Edition__Leder__T...   3090       1   
233721  VW_Golf_5___1.9_TDI____Navigation_PDC_Partieke...   3900       0   
257254  Volkswagen_Golf_1.9_TDI_schekheftgepflegt_T�V_...   3750       1   
259845  Volkswagen_Volkswagen_Golf_5_1.9TDI_DPF_8_fach...   7800       0   
263685                          Volkswagen_Golf_5_2.0_TDI   3250       0   
266602                        Volkswagen_Golf_1.6_Edition   7990       0   
278009                   Volkswagen_Golf__TDI_Comfortline   7900       0   
282082                        Volkswagen_Golf_1.6_Edition   3750       0   
289346                Volkswagen_Golf_1.9_TDI_VOLLERLEDER   3500       0   
295282  Golf_5_1_9l_TDI_Zahnriemen_neu_T�V_08/2017_top...   4600       0   
296335  Volkswagen_Golf_1.9_TDI_18Zoll_Eibach_Gewindef...   3200       0   
328378                    Volkswagen_Golf_1.9_TDI_Edition   3100       1   
331592     Volkswagen_Golf_1.9_TDI_Euro_4_Gruene_Plakette   3500       0   
333301                        Volkswagen_Golf_1.6_Edition   4990       0   
335758  Volkswagen_Golf_1.6_"Team"_Climatronic__Alu__E...   9950       0   
337468                        Volkswagen_Golf_1.6_Edition   5990       0   
347447                    Volkswagen_Golf_1.9_TDI_Edition   5500       1   
351297                    Volkswagen_Golf_1.9_TDI_Edition   3700       0   
356393        Volkswagen_Golf_1.9_TDI_Comfortline_T�V_neu   3800       1   
362342                    Volkswagen_Golf_1.9_TDI_Edition   3699       0   
363770                   